In [6]:
import pandas as pd
from datetime import datetime

# Load cleaned dataset
df = pd.read_csv("vehicles_cleaned.csv")

# Set constants
USD_TO_CAD = 1.35
MILES_TO_KM = 1.60934

# Convert price and odometer
df['price'] = (df['price'] * USD_TO_CAD).round(0).astype("Int64")
df['odometer'] = (df['odometer'] * MILES_TO_KM).round(0).astype("Int64")

# Normalize text fields
text_fields = ['manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'transmission', 'drive', 'size', 'type', 'paint_color']
available_fields = [col for col in text_fields if col in df.columns]

for col in available_fields:
    df[col] = df[col].astype(str).str.strip().str.lower().replace("nan", None)

# Drop irrelevant columns
columns_to_drop = ['url', 'region_url', 'image_url', 'county', 'VIN']
available_to_drop = [col for col in columns_to_drop if col in df.columns]
df.drop(columns=available_to_drop, inplace=True)

# Add created_at
df['created_at'] = datetime.now().strftime("%Y-%m-%d")

# Preview
df.head()

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,state,created_at
0,45346,2014.0,gmc,sierra 1500 crew cab slt,good,8 cylinders,gas,93218,clean,other,None,pickup,white,al,2025-05-26
1,30497,2010.0,chevrolet,silverado 1500,good,8 cylinders,gas,114632,clean,other,None,pickup,blue,al,2025-05-26
2,53446,2020.0,chevrolet,silverado 1500 crew,good,8 cylinders,gas,30835,clean,other,None,pickup,red,al,2025-05-26
3,41836,2017.0,toyota,tundra double cab sr,good,8 cylinders,gas,66182,clean,other,None,pickup,red,al,2025-05-26
4,20250,2013.0,ford,f-150 xlt,excellent,6 cylinders,gas,205996,clean,automatic,rwd,truck,black,al,2025-05-26


In [8]:
original_df = pd.read_csv("vehicles.csv")
original_df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')

In [9]:
original_df = pd.read_csv("vehicles.csv")
original_df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')

In [10]:
# Load original dataset with posting_date
original_df = pd.read_csv("vehicles.csv")

# Replace created_at with posting_date from original
df['created_at'] = pd.to_datetime(original_df['posting_date'], errors='coerce').dt.date

C:\Users\huynha8\AppData\Local\Temp\ipykernel_27424\272815841.py:5: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['created_at'] = pd.to_datetime(original_df['posting_date'], errors='coerce').dt.date


AttributeError: Can only use .dt accessor with datetimelike values

In [12]:
# Load original dataset
original_df = pd.read_csv("vehicles.csv")

# Try parsing posting_date with error fallback
posting_dates = pd.to_datetime(original_df['posting_date'], errors='coerce', utc=True)

# Now extract the date part safely
df['created_at'] = posting_dates.dt.date

In [13]:
# Load original dataset
original_df = pd.read_csv("vehicles.csv")

# Try parsing posting_date with error fallback
posting_dates = pd.to_datetime(original_df['posting_date'], errors='coerce', utc=True)

# Now extract the date part safely
df['created_at'] = posting_dates.dt.date

In [14]:
df[['model', 'price', 'created_at']].head()

,model,price,created_at
0,sierra 1500 crew cab slt,45346,NaT
1,silverado 1500,30497,NaT
2,silverado 1500 crew,53446,NaT
3,tundra double cab sr,41836,NaT
4,f-150 xlt,20250,NaT


In [15]:
# Load original dataset with posting_date
original_df = pd.read_csv("vehicles.csv", usecols=["id", "posting_date"])

# Convert posting_date to datetime safely
original_df['posting_date'] = pd.to_datetime(original_df['posting_date'], errors='coerce', utc=True).dt.date

# Merge posting_date into df using 'id'
df = df.merge(original_df, how='left', left_on='id', right_on='id')

# Rename for clarity
df.rename(columns={'posting_date': 'created_at'}, inplace=True)

KeyError: 'id'

In [16]:
print("df columns:", df.columns)
print("original_df columns:", original_df.columns)

df columns: Index(['price', 'year', 'manufacturer', 'model', 'condition', 'cylinders',
       'fuel', 'odometer', 'title_status', 'transmission', 'drive', 'type',
       'paint_color', 'state', 'created_at'],
      dtype='object')
original_df columns: Index(['id', 'posting_date'], dtype='object')


In [17]:
df = pd.read_csv("vehicles_cleaned.csv")  # This version must include 'id'

In [18]:
print("df columns:", df.columns)

df columns: Index(['price', 'year', 'manufacturer', 'model', 'condition', 'cylinders',
       'fuel', 'odometer', 'title_status', 'transmission', 'drive', 'type',
       'paint_color', 'state'],
      dtype='object')


In [19]:
# Only keep what's needed from original
original_df = pd.read_csv("vehicles.csv", usecols=["id", "posting_date"])
original_df['posting_date'] = pd.to_datetime(original_df['posting_date'], errors='coerce', utc=True).dt.date

# Merge on id
df = df.merge(original_df, how='left', on='id')
df.rename(columns={'posting_date': 'created_at'}, inplace=True)

KeyError: 'id'

In [20]:
# Load cleaned dataset again from scratch, but include 'id'
df = pd.read_csv("vehicles.csv")

# Keep only columns we care about (including 'id' now)
columns_to_keep = [
    'id', 'price', 'year', 'manufacturer', 'model', 'condition',
    'cylinders', 'fuel', 'odometer', 'title_status',
    'transmission', 'drive', 'type', 'paint_color', 'state'
]
df = df[columns_to_keep].copy()

In [21]:
# Convert currency and odometer to Canadian units
USD_TO_CAD = 1.35
MILES_TO_KM = 1.60934

df['price'] = (df['price'] * USD_TO_CAD).round(0).astype("Int64")
df['odometer'] = (df['odometer'] * MILES_TO_KM).round(0).astype("Int64")

# Normalize text fields if they exist
text_fields = ['manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'transmission', 'drive', 'type', 'paint_color']
for col in text_fields:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.lower().replace("nan", None)

In [22]:
# Extract and convert posting_date from original
original_df = pd.read_csv("vehicles.csv", usecols=["id", "posting_date"])
original_df['posting_date'] = pd.to_datetime(original_df['posting_date'], errors='coerce', utc=True).dt.date

# Merge on id
df = df.merge(original_df, how='left', on='id')
df.rename(columns={'posting_date': 'created_at'}, inplace=True)

In [23]:
df[['id', 'model', 'price', 'created_at']].head()

,id,model,price,created_at
0,7222695916,None,8100,NaT
1,7218891961,None,16065,NaT
2,7221797935,None,28350,NaT
3,7222270760,None,2025,NaT
4,7210384030,None,6615,NaT


In [24]:
df[['model', 'created_at']].isna().sum()

model         5277
created_at      68
dtype: int64

In [25]:
df[df['model'].isna()].head()
df[df['created_at'].isna()].head()

,id,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,state,created_at
0,7222695916,8100,NaN,None,None,None,None,None,<NA>,NaN,None,None,None,None,az,NaT
1,7218891961,16065,NaN,None,None,None,None,None,<NA>,NaN,None,None,None,None,ar,NaT
2,7221797935,28350,NaN,None,None,None,None,None,<NA>,NaN,None,None,None,None,fl,NaT
3,7222270760,2025,NaN,None,None,None,None,None,<NA>,NaN,None,None,None,None,ma,NaT
4,7210384030,6615,NaN,None,None,None,None,None,<NA>,NaN,None,None,None,None,nc,NaT


In [26]:
raw_df = pd.read_csv("vehicles.csv", usecols=["id", "model", "posting_date"])
raw_df[raw_df['model'].isna()].shape

(5277, 3)

In [27]:
raw_df[raw_df['model'].isna()].head()

,id,model,posting_date
0,7222695916,NaN,NaN
1,7218891961,NaN,NaN
2,7221797935,NaN,NaN
3,7222270760,NaN,NaN
4,7210384030,NaN,NaN


In [28]:
print(f"Final row count: {len(df)}")

Final row count: 426880


In [29]:
df = df.dropna(subset=['model', 'created_at'])

In [30]:
print(f"Final row count: {len(df)}")

Final row count: 421603


In [31]:
df.to_csv("vehicles_final_cleaned.csv", index=False)

In [32]:
import sqlite3

# Load your final cleaned CSV
df = pd.read_csv("vehicles_final_cleaned.csv")

# Connect to your database (will create it if not exists)
conn = sqlite3.connect("../car_reference.db")  # adjust path if needed

# Write to a new table
df.to_sql("reference_listings", conn, if_exists="replace", index=False)

# Confirm
print("✅ Data imported into reference_listings")
conn.close()

✅ Data imported into reference_listings


In [33]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect("../car_reference.db")

In [34]:
# Preview 5 rows
pd.read_sql_query("SELECT id, make, model, year, price, odometer, created_at FROM reference_listings LIMIT 5", conn)

DatabaseError: Execution failed on sql 'SELECT id, make, model, year, price, odometer, created_at FROM reference_listings LIMIT 5': no such column: make

In [35]:
pd.read_sql_query("PRAGMA table_info(reference_listings)", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,0
1,1,price,INTEGER,0,None,0
2,2,year,REAL,0,None,0
3,3,manufacturer,TEXT,0,None,0
4,4,model,TEXT,0,None,0
5,5,condition,TEXT,0,None,0
6,6,cylinders,TEXT,0,None,0
7,7,fuel,TEXT,0,None,0
8,8,odometer,REAL,0,None,0
9,9,title_status,TEXT,0,None,0


In [36]:
pd.read_sql_query("SELECT id, manufacturer, model, year, price, odometer, created_at FROM reference_listings LIMIT 5", conn)

,id,manufacturer,model,year,price,odometer,created_at
0,7316814884,gmc,sierra 1500 crew cab slt,2014.0,45346,93218.0,2021-05-04
1,7316814758,chevrolet,silverado 1500,2010.0,30497,114632.0,2021-05-04
2,7316814989,chevrolet,silverado 1500 crew,2020.0,53446,30835.0,2021-05-04
3,7316743432,toyota,tundra double cab sr,2017.0,41836,66182.0,2021-05-04
4,7316356412,ford,f-150 xlt,2013.0,20250,205996.0,2021-05-03
